In [6]:
# https://runawayhorse001.github.io/LearningApacheSpark/clustering.html
from utilsom import * 

In [7]:
input_data = spark.read.csv("Shuttle_withoutdupl_norm_v06.csv", header=True, inferSchema=True, mode="DROPMALFORMED", encoding='UTF-8')
#df = df.drop("_c0")
#input_data.show(n=5,truncate=False)
#http://archive.ics.uci.edu/ml/datasets/HEPMASS  
#input_data = load_data("hdfs://localhost:9000/data/htru.csv")

print(input_data.count())
print(input_data)

1013
DataFrame[_c0: int, att1: double, att2: double, att3: double, att4: double, att5: double, att6: double, att7: double, att8: double, att9: double, id: double, outlier: string]


In [8]:


# dellet label
df_input_data = input_data.drop("_c0","outlier","id")

# staep 2
df_input_data = shuffling(df_input_data, 10)

# staep 3
input_features = assembler(df_input_data,df_input_data.columns[0:])

# staep 4
scaledData = scaler(input_features)

# SOM

In [38]:
#https://machinelearningnepal.com/2018/01/22/apache-spark-implementation-of-som-batch-algorithm/
# step 5 Som
from som.batch_som import SOM
from matplotlib import pyplot as plt
from matplotlib import patches as patches

mapsize = calculate_map_size(input_data)
print('Som: X = {} and Y = {}, {} neurones'.format(mapsize[0], mapsize[1],mapsize[0]*mapsize[1]))

#x =  mapsize[0] 
#y = mapsize[1]
som = SOM(18,20,9) # (x×y) neurons, (70 features)
som.train(scaledData, 10) # data , epochs  +10

   # for show All codebook
#som.net
   # dimention of codebook (x,y,features)
print('dimention of codebook 3D:',som.net.shape)

#som.net.reshpae(x*y,features)
data = som.net
data = data.transpose(2,1,0).reshape(-1,data.shape[2])
print('dimention of codebook 2D:',data.shape)

C:\Users\Principal\Desktop\these\SOM\SomSpark\som chine\Shuttle\utilsom.py:81: RuntimeWarning: invalid value encountered in add
  A = np.ndarray(shape=[dim, dim]) + np.Inf


Som: X = 14 and Y = 11, 154 neurones
SOM training epoches 1
neighborhood radius  13.481718494401385
-------------------------------------
SOM training epoches 2
neighborhood radius  10.097596309015795
-------------------------------------
SOM training epoches 3
neighborhood radius  7.562941717125411
-------------------------------------
SOM training epoches 4
neighborhood radius  5.664525067769413
-------------------------------------
SOM training epoches 5
neighborhood radius  4.242640687119286
-------------------------------------
SOM training epoches 6
neighborhood radius  3.1776715231464365
-------------------------------------
SOM training epoches 7
neighborhood radius  2.3800262745964416
-------------------------------------
SOM training epoches 8
neighborhood radius  1.7826024579660036
-------------------------------------
SOM training epoches 9
neighborhood radius  1.3351413625403128
-------------------------------------
SOM training epoches 10
neighborhood radius  1.0000000000

In [39]:
# prediction som
# pour avoir bmu, bmu, index 
pre = som.predict(scaledData)
pre.show(2)
print(pre.count())

+---------------+--------------+--------------+---------------+--------------+---------------+--------------+--------------+--------------+--------------------+---------+--------------------+
|           att1|          att2|          att3|           att4|          att5|           att6|          att7|          att8|          att9|                 bmu|  bmu_idx|            features|
+---------------+--------------+--------------+---------------+--------------+---------------+--------------+--------------+--------------+--------------------+---------+--------------------+
|0.0114942528736|0.272727272727|0.702127659574|0.0519761775853|0.193181818182|0.0121744791667|0.923076923077|0.843260188088|0.711340206186|[0.01267699824735...|[1.0,5.0]|[0.0114942528736,...|
|0.0114942528736|0.251336898396|0.404255319149|0.0508933405522|0.136363636364|0.0127604166667| 0.78021978022|0.865203761755|0.762886597938|[0.01663534969058...|[4.0,0.0]|[0.0114942528736,...|
+---------------+--------------+--------

In [40]:
# to kmenas, select, bmu 
bmux = pre.select('bmu','bmu_idx','features')
print(bmux.show(5))

+--------------------+-----------+--------------------+
|                 bmu|    bmu_idx|            features|
+--------------------+-----------+--------------------+
|[0.01267699824735...|  [1.0,5.0]|[0.0114942528736,...|
|[0.01663534969058...|  [4.0,0.0]|[0.0114942528736,...|
|[0.21484256847656...| [8.0,17.0]|[0.218390804598,0...|
|[0.18673934652225...|[10.0,14.0]|[0.183908045977,0...|
|[0.01276592229651...| [16.0,4.0]|[0.0114942528736,...|
+--------------------+-----------+--------------------+
only showing top 5 rows

None


# K-means

In [41]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="bmu", \
                               outputCol="indexedbmu",\
                               ).fit(bmux)

data = featureIndexer.transform(bmux)

In [42]:
data.show(5,True)

+--------------------+-----------+--------------------+--------------------+
|                 bmu|    bmu_idx|            features|          indexedbmu|
+--------------------+-----------+--------------------+--------------------+
|[0.01267699824735...|  [1.0,5.0]|[0.0114942528736,...|[0.01267699824735...|
|[0.01663534969058...|  [4.0,0.0]|[0.0114942528736,...|[0.01663534969058...|
|[0.21484256847656...| [8.0,17.0]|[0.218390804598,0...|[0.21484256847656...|
|[0.18673934652225...|[10.0,14.0]|[0.183908045977,0...|[0.18673934652225...|
|[0.01276592229651...| [16.0,4.0]|[0.0114942528736,...|[0.01276592229651...|
+--------------------+-----------+--------------------+--------------------+
only showing top 5 rows



In [43]:
from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans() \
          .setK(2) \
          .setFeaturesCol("indexedbmu")\
          .setPredictionCol("cluster")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, kmeans])

model = pipeline.fit(bmux)

cluster = model.transform(bmux)

In [44]:
cluster = model.transform(bmux)

In [45]:
cluster.show(5)

+--------------------+-----------+--------------------+--------------------+-------+
|                 bmu|    bmu_idx|            features|          indexedbmu|cluster|
+--------------------+-----------+--------------------+--------------------+-------+
|[0.01267699824735...|  [1.0,5.0]|[0.0114942528736,...|[0.01267699824735...|      1|
|[0.01663534969058...|  [4.0,0.0]|[0.0114942528736,...|[0.01663534969058...|      1|
|[0.21484256847656...| [8.0,17.0]|[0.218390804598,0...|[0.21484256847656...|      1|
|[0.18673934652225...|[10.0,14.0]|[0.183908045977,0...|[0.18673934652225...|      1|
|[0.01276592229651...| [16.0,4.0]|[0.0114942528736,...|[0.01276592229651...|      1|
+--------------------+-----------+--------------------+--------------------+-------+
only showing top 5 rows



In [46]:
y_pred = cluster.select('cluster')

In [47]:
import numpy as np
y_pred = np.array(list(cluster.select('cluster').toPandas()['cluster']))

In [48]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
# load data

df = pd.read_csv("Shuttle_withoutdupl_norm_v06.csv")
del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

# df to values
df = df.values
# 
y_test = df[:,9]

In [49]:
# predict
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of SOM K-means = {:.6f} %".format(acc * 100))
# Matrix de confusion
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

Test Accuracy of SOM K-means = 15.301086 %
[[144 856]
 [  2  11]]


              precision    recall  f1-score   support

         0.0       0.99      0.14      0.25      1000
         1.0       0.01      0.85      0.02        13

   micro avg       0.15      0.15      0.15      1013
   macro avg       0.50      0.50      0.14      1013
weighted avg       0.97      0.15      0.25      1013

